In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
from time import time
from hierarchical_classifier import HierarchicalClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import joblib

# Читаем данные, выкидывая дубликаты и выбирая только нужные нам колонки

In [3]:
data_path_train = './amazon/train_40k.csv'
data_path_test = './amazon/val_10k.csv'

train_df = pd.read_csv(data_path_train)
train_df = train_df[['Text', 'Cat1', 'Cat2', 'Cat3']].drop_duplicates(ignore_index=True)
test_df = pd.read_csv(data_path_test).drop_duplicates()
test_df = test_df[['Text', 'Cat1', 'Cat2', 'Cat3']].drop_duplicates(ignore_index=True)

In [4]:
train_df.head()

,Text,Cat1,Cat2,Cat3
0,The description and photo on this product need...,grocery gourmet food,meat poultry,jerky
1,This was a great book!!!! It is well thought t...,toys games,games,unknown
2,"I am a first year teacher, teaching 5th grade....",toys games,games,unknown
3,I got the book at my bookfair at school lookin...,toys games,games,unknown
4,Hi! I'm Martine Redman and I created this puzz...,toys games,puzzles,jigsaw puzzles


In [5]:
test_df.head()

,Text,Cat1,Cat2,Cat3
0,We've only had it installed about 2 weeks. So ...,pet supplies,cats,cat flaps
1,My bunny had a hard time eating this because t...,pet supplies,bunny rabbit central,food
2,would never in a million years have guessed th...,health personal care,health care,massage relaxation
3,"Being the jerky fanatic I am, snackmasters han...",grocery gourmet food,snack food,jerky dried meats
4,Wondered how quick my dog would catch on to th...,pet supplies,dogs,toys


# Убедимся, что в данных нет пропусков, и посмотрим, сколько уникальных значений есть в каждой колонке

In [6]:
train_df.isna().sum()

Text    0
Cat1    0
Cat2    0
Cat3    0
dtype: int64

In [7]:
train_df.nunique()

Text    39489
Cat1        6
Cat2       64
Cat3      464
dtype: int64

In [8]:
test_df.isna().sum()

Text    0
Cat1    0
Cat2    0
Cat3    0
dtype: int64

In [9]:
test_df.nunique()

Text    9854
Cat1       6
Cat2      64
Cat3     377
dtype: int64

# Создадим колонку, в которой объединены все три категории, чтобы затем обучить на ней плоский классификатор

In [10]:
train_df['Cat1Cat2Cat3'] = train_df['Cat1'] + '/' + train_df['Cat2'] + '/' + train_df['Cat3']
test_df['Cat1Cat2Cat3'] = test_df['Cat1'] + '/' + test_df['Cat2'] + '/' + test_df['Cat3']
train_df['Cat1Cat2Cat3'].nunique()

555

In [11]:
train_df.head()

,Text,Cat1,Cat2,Cat3,Cat1Cat2Cat3
0,The description and photo on this product need...,grocery gourmet food,meat poultry,jerky,grocery gourmet food/meat poultry/jerky
1,This was a great book!!!! It is well thought t...,toys games,games,unknown,toys games/games/unknown
2,"I am a first year teacher, teaching 5th grade....",toys games,games,unknown,toys games/games/unknown
3,I got the book at my bookfair at school lookin...,toys games,games,unknown,toys games/games/unknown
4,Hi! I'm Martine Redman and I created this puzz...,toys games,puzzles,jigsaw puzzles,toys games/puzzles/jigsaw puzzles


In [12]:
test_df.head()

,Text,Cat1,Cat2,Cat3,Cat1Cat2Cat3
0,We've only had it installed about 2 weeks. So ...,pet supplies,cats,cat flaps,pet supplies/cats/cat flaps
1,My bunny had a hard time eating this because t...,pet supplies,bunny rabbit central,food,pet supplies/bunny rabbit central/food
2,would never in a million years have guessed th...,health personal care,health care,massage relaxation,health personal care/health care/massage relax...
3,"Being the jerky fanatic I am, snackmasters han...",grocery gourmet food,snack food,jerky dried meats,grocery gourmet food/snack food/jerky dried meats
4,Wondered how quick my dog would catch on to th...,pet supplies,dogs,toys,pet supplies/dogs/toys


# Обучим иерархический классификатор с базовой моделью логистической регрессии и посмотрим на его предсказания на тестовой выборке

In [13]:
logreg_cls = HierarchicalClassifier(LogisticRegression, max_iter=500)
logreg_cls.fit(train_df['Text'], train_df[['Cat1', 'Cat2', 'Cat3']].to_numpy())
y_predicted_logreg = logreg_cls.predict(test_df['Text'])

# Посмотрим, какие получаются метрики (их для удобства считаем с помощью sklearn.metrics.classification_report так как он сразу выдаёт все нужные показатели - для каждого класса соответствующие precision, recall, f1, accuracy, а также результаты macro-усреднения)

In [14]:
print(metrics.classification_report(test_df['Cat1'], y_predicted_logreg[:, 0]))

                      precision    recall  f1-score   support

       baby products       0.68      0.65      0.67       697
              beauty       0.86      0.78      0.82      2073
grocery gourmet food       0.82      0.70      0.76       833
health personal care       0.72      0.85      0.78      2942
        pet supplies       0.94      0.78      0.85      1566
          toys games       0.81      0.85      0.83      1758

            accuracy                           0.80      9869
           macro avg       0.81      0.77      0.78      9869
        weighted avg       0.81      0.80      0.80      9869



# Видим, что все 6 категорий предсказываются довольно неплохо. Само собой, есть корреляция между количеством данных в данной категории и качеством предсказаний - чем меньше данных, тем хуже модель обучается на них, и тем хуже предсказания.

In [15]:
print(metrics.classification_report(test_df['Cat2'], y_predicted_logreg[:, 1]))

                                  precision    recall  f1-score   support

              action toy figures       0.22      0.54      0.31       114
                     arts crafts       0.57      0.24      0.34        99
                 baby child care       0.50      0.05      0.10        37
                       baby food       0.00      0.00      0.00         2
               baby toddler toys       0.33      0.50      0.40       139
                       bath body       0.58      0.27      0.37       140
               bathing skin care       0.57      0.17      0.27        46
                       beverages       0.68      0.66      0.67       215
                           birds       0.91      0.20      0.32        51
                   breads bakery       0.40      0.09      0.15        22
                 breakfast foods       0.50      0.23      0.32        39
                   building toys       0.61      0.48      0.54        89
            bunny rabbit central     

C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
print(metrics.classification_report(test_df['Cat3'], y_predicted_logreg[:, 2]))

                                    precision    recall  f1-score   support

                       accessories       0.26      0.19      0.22        37
                action toy figures       0.00      0.00      0.00         3
     activity centers entertainers       0.33      0.18      0.24        11
             activity play centers       0.00      0.00      0.00         2
                  adult toys games       1.00      0.08      0.15        48
                    air fresheners       0.00      0.00      0.00         8
                            albums       0.00      0.00      0.00         2
                           allergy       0.56      0.19      0.29        26
              alternative medicine       0.16      0.18      0.17        74
                   animals figures       0.37      0.50      0.42       139
               apparel accessories       1.00      0.40      0.57        30
                     aprons smocks       0.00      0.00      0.00         3
           

C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
print(metrics.classification_report(test_df['Cat1Cat2Cat3'], y_predicted_logreg[:, 0] + '/' + y_predicted_logreg[:, 1] + '/' + y_predicted_logreg[:, 2]))

                                                                                    precision    recall  f1-score   support

                                baby products/bathing skin care/bathing tubs seats       0.33      0.50      0.40         8
                          baby products/bathing skin care/grooming healthcare kits       0.00      0.00      0.00         7
                                           baby products/bathing skin care/shampoo       0.00      0.00      0.00         1
                                         baby products/bathing skin care/skin care       0.50      0.08      0.13        13
                                   baby products/bathing skin care/soaps cleansers       0.00      0.00      0.00         6
                                           baby products/bathing skin care/unknown       0.00      0.00      0.00        11
                                   baby products/car seats accessories/accessories       0.38      0.33      0.35        18
       

C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Видим, что для Cat1 результат получился не самым плохим, а для Cat2, Cat3 - похуже.
# Возможно, с Cat3 ситуация совсем плохая, так как для предсказания Cat3 при данной паре (Cat1, Cat2) остаётся слишком мало данных, и модель не может хорошо понять распределение

# Теперь посмотрим, что получается при обучении иерархического классификатора со случайным лесом в качестве базовой модели.

In [21]:
rf_cls = HierarchicalClassifier(RandomForestClassifier, n_jobs=-1)
rf_cls.fit(train_df['Text'], train_df[['Cat1', 'Cat2', 'Cat3']].to_numpy())
y_predicted_rf = rf_cls.predict(test_df['Text'])

# Посмотрим на метрики

In [22]:
print(metrics.classification_report(test_df['Cat1'], y_predicted_rf[:, 0]))

                      precision    recall  f1-score   support

       baby products       0.70      0.49      0.58       697
              beauty       0.82      0.67      0.74      2073
grocery gourmet food       0.75      0.54      0.63       833
health personal care       0.60      0.79      0.68      2942
        pet supplies       0.93      0.66      0.77      1566
          toys games       0.67      0.80      0.73      1758

            accuracy                           0.70      9869
           macro avg       0.75      0.66      0.69      9869
        weighted avg       0.73      0.70      0.70      9869



In [23]:
print(metrics.classification_report(test_df['Cat2'], y_predicted_rf[:, 1]))

                                  precision    recall  f1-score   support

              action toy figures       0.16      0.44      0.23       114
                     arts crafts       0.68      0.17      0.27        99
                 baby child care       0.00      0.00      0.00        37
                       baby food       0.00      0.00      0.00         2
               baby toddler toys       0.18      0.45      0.26       139
                       bath body       0.55      0.13      0.21       140
               bathing skin care       0.83      0.11      0.19        46
                       beverages       0.66      0.67      0.67       215
                           birds       1.00      0.02      0.04        51
                   breads bakery       0.25      0.05      0.08        22
                 breakfast foods       0.32      0.23      0.27        39
                   building toys       0.69      0.42      0.52        89
            bunny rabbit central     

C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
print(metrics.classification_report(test_df['Cat3'], y_predicted_rf[:, 2]))

                                    precision    recall  f1-score   support

                       accessories       0.31      0.24      0.27        37
                action toy figures       0.00      0.00      0.00         3
     activity centers entertainers       0.00      0.00      0.00        11
             activity play centers       0.00      0.00      0.00         2
                  adult toys games       0.50      0.04      0.08        48
                    air fresheners       0.00      0.00      0.00         8
                            albums       0.00      0.00      0.00         2
                           allergy       0.48      0.50      0.49        26
              alternative medicine       0.11      0.20      0.14        74
                   animals figures       0.27      0.36      0.31       139
               apparel accessories       0.89      0.27      0.41        30
                     aprons smocks       0.00      0.00      0.00         3
           

C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: 

# Видно, что иерархический классификатор со случайным лесом в качестве базовой модели в данном случае справился хуже, чем иерархический классификатор с логистической регрессией. Возможно, это может измениться при подборе гиперпараметров базовых моделей.

# Обучим теперь плоский классификатор на основе логистической регрессии и сравним его с иерархическим классификатором

In [ ]:
flat_logreg_cls = Pipeline([
            ('tf-idf', TfidfVectorizer(max_features=10000)),
            ('clf', LogisticRegression(max_iter=500))
        ])
flat_logreg_cls.fit(train_df['Text'], train_df['Cat1Cat2Cat3'])
y_predicted_flat_logreg = flat_logreg_cls.predict(test_df['Text'])

In [33]:
print(metrics.classification_report(test_df['Cat1Cat2Cat3'], y_predicted_flat_logreg))

                                                                                    precision    recall  f1-score   support

                                baby products/bathing skin care/bathing tubs seats       0.15      0.25      0.19         8
                          baby products/bathing skin care/grooming healthcare kits       0.00      0.00      0.00         7
                                           baby products/bathing skin care/shampoo       0.00      0.00      0.00         1
                                         baby products/bathing skin care/skin care       0.00      0.00      0.00        13
                                   baby products/bathing skin care/soaps cleansers       0.00      0.00      0.00         6
                                           baby products/bathing skin care/unknown       0.00      0.00      0.00        11
                                   baby products/car seats accessories/accessories       0.20      0.22      0.21        18
       

C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Progi\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Видим, что плоский классификатор в данном случае даёт то же качество, что и иерархический - единственное маленькое отличие наблюдается в accuracy (0.41 у иерархического и 0.42 у плоского) и macro-averaged precision (0.20 у иерархического и 0.19 у плоского).
# При этом, как в случае с иерархическим классификатором, довольно много категорий по сути не предсказываются вообще (precision=recall=f1=0). Видимо, это связано с тем, что модели всё же не хватило данных на такое большое количество классов.

# Теперь сделаем профайлинг иерархического и плоского классификаторов с логистической регрессией в виде базовой модели.
# Сначала я пытался сделать профайлинг с помощью средств PyCharm и pyinstruments, однако они профайлят именно функции, что в данном случае тяжело воспринимается и не очень информативно, так как выдаётся информация о внутренних функциях sklearn (по типу солвера LBFGS для логистической регрессии). Поэтому я решил использовать line_profiler - он выдаёт информацию о каждой строке кода интересующей функции.
# Я отдельно запрофайлил методы fit и predict иерархического классификатора, а также плоский классификатор (профайлинг я делал на линуксе, т.к. в винде он почему-то отказался нормально работать).

# Посмотрим на результат:

In [14]:
with open('profile_output.txt', encoding = 'utf-8') as f:
    contents = f.read()
    print(contents)

Timer unit: 1e-09 s

Total time: 13.7363 s
File: /home/vk/samokat/hierarchical_classifier.py
Function: predict at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                               @line_profiler.profile
    77                                               def predict(self, X):
    78                                                   """
    79                                                   Предсказание иерархического классификатора.
    80                                           
    81                                                   :param X: Тексты.
    82                                                   :return: Предсказанные метки классов (Cat1, Cat2, Cat3).
    83                                                   """
    84                                                   # Сначала предсказывается Cat1 для всех объектов X
    85         1  520679075.0    5e+08      3.8          y_pred_cat1 = self.model_cat1.predic

# Видим, что в методе fit иерархического классификатора наибольшее время занимает обучение моделей для предсказания Cat3 внутри вложенного цикла for, затем чуть меньше времени занимает обучение моделей для предсказания Cat2, и наименьшее время - предсказание Cat1.
# Ускорить обучение модели для предсказания Cat1, наверное, особо не получится. Что касается ускорения циклов для предсказания Cat2 и Cat3, то тут, наверное, можно использовать multiprocessing, ведь мы совершенно независимо обучаем модели при данных Cat1 или (Cat1, Cat2) - эти независимые процессы и можно делегировать multiprocessing. Либо можно попытаться обернуть циклы в numba.

# В методе predict иерархического классификатора дольше всего занимает предсказание Cat2 и Cat3 внутри цикла. Это происходит довольно долго, так как здесь мы действуем последовательно, проходясь по всей тестовой выборке - сначала предсказываем Cat1 для всех объектов выборки, а уже затем на основе предсказанных Cat1 вызываем подходящие модели для предсказания Cat2 (и далее для Cat3). Возможно, это можно ускорить путём группировки предсказаний - выделять из тестовой выборки объекты, для которых мы предсказали конкретное значение Cat1 и обрабатывать их все вместе, предсказывая Cat2 (и затем похожим образом поступать с Cat3).

# В плоском классификаторе основное время занимает именно метод fit, а метод predict на его фоне совершенно незаметен.

# Кроме того, уже у себя на ноутбуке на Windows с помощью обычного python пакета time я посмотрел общее время выполенения методов fit и predict иерархического классификатора и плоского классификатора.
# Метод fit иерархического классификатора: 29.1 с
# Метод predict иерархического классификатора: 11.3 с
# fit + predict плоского классификатора: 142.3 с
# Видим, что плоский классификатор учится значительно дольше, но предсказывает значительно быстрее.

# Наконец, сохраним обученный иерархический классификатор с логистической регрессией в качестве базовой модели ()

In [14]:
logreg_cls.save_model("hierarchical_logreg_cls_model.pkl")

Модель сохранена в hierarchical_logreg_cls_model.pkl!


# Проверим, что всё работает (конечно, можно загрузить и с помощью joblib.load):

In [15]:
loaded_logreg_cls = HierarchicalClassifier.load_model("hierarchical_logreg_cls_model.pkl")
predictions = loaded_logreg_cls.predict(test_df['Text'])

Модель загружена из hierarchical_logreg_cls_model.pkl!


In [16]:
predictions

array([['pet supplies', 'dogs', 'toys'],
       ['pet supplies', 'dogs', 'toys'],
       ['toys games', 'electronics for kids', 'unknown'],
       ...,
       ['pet supplies', 'dogs', 'beds furniture'],
       ['pet supplies', 'dogs', 'toys'],
       ['health personal care', 'health care', 'pain relievers']],
      dtype=object)

# Небольшие рассуждения по поводу иерархического классификатора
## Здесь я использовал подход с условными распределениями. В целом, можно, например, не плодить большое количество классификаторов и сделать всего три независимых модели на каждую категорию - первый классификатор предсказывает Cat1, второй - Cat2, третий Cat3. Однако математического обоснования корректности такого подхода у меня нет. Зато оно, возможно, работало бы быстрее и потребовало бы меньше кода, который при этом был бы проще.

## P.S. Кроме того, наверное, можно также сделать иерархический классификатор для произвольного числа категорий (Cat1, Cat2, ..., Catk) с использованием рекурсии или стека, храня текущий префикс категорий и не затачиваясь под фиксированное количество категорий (ведь, в целом, в будущем в магазине могут добавиться новые товары и новые категории).

## P.P.S Ещё я пробовал немного почистить и обработать данные с помощью функции ниже, но это особо не дало никакого прироста метрик.

In [ ]:
import pandarallel
pandarallel.initialize()

def clear_data(df):
    import re
    import nltk
    nltk.download('wordnet')
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer

    lemmatizer = WordNetLemmatizer()
    df['Text'] = df['Text'].parallel_apply(lambda s: s.strip().split('|')[-1].split('http')[0].strip().lower())
    df['Text'] = df['Text'].parallel_apply(lambda text: re.sub(r'[^A-Za-z\s]+', '', text))
    df['Text'] = df['Text'].parallel_apply(lambda text: word_tokenize(text))
    df['Text'] = df['Text'].parallel_apply(lambda text: [lemmatizer.lemmatize(word) for word in text])
    df['Text'] = df['Text'].parallel_apply(lambda text: ' '.join(text))

    return df